In [1]:
from bs4 import BeautifulSoup
import csv
import re
import os 
import pandas as pd
import glob

In [2]:
data = []

# reading html files from folder

#'Leo/Riverside/Riv_Shelter_HTML_Files'is the name for the shelter HTML Files Riverside
#'Leo/Riverside/Riv_Food_Pantry_HTML_Files'is the name for the food pantry HTML Files for Riverside 
#'Leo/Riverside/Riv_Mental_Health_HTML_Files'is the name for the mental health HTML Files for Riverside
#'Leo/Downtown_LA/Downtown_LA_Food_Pantry_HTML_Files' is the name for the food pantry HTML Files for downtown LA
#'Leo/Downtown_LA/Downtown_LA_Shelter_HTML_Files'
#'Leo/Downtown_LA/Downtown_LA_Mental_Health_HTML_Files'
#'Irvine_Mental_Health_HTML_Files' is the name for the mental health HTML Files for Irvine
folder_path = 'Leo/Downtown_LA/Downtown_LA_Food_Pantry_HTML_Files'

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.html'):
        file_path = os.path.join(folder_path, filename)

        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        soup = BeautifulSoup(html_content, 'html.parser')


        # Find the <ul> element with class "best-programs"
        ul_element = soup.find('ul', {'class': 'best-programs'})

        # Find all <li> elements within the <ul>
        li_elements = ul_element.find_all('li', {'class': 'search-result card card-v3 program-info'})

        # Loop through the found <li> elements and extract labels and values
        for li_element in li_elements:
            # Use BeautifulSoup to parse the HTML of each li_element
            li_soup = BeautifulSoup(str(li_element), 'html.parser')

            # Find the program heading 
            program_heading_element = li_soup.find('div', {'class': 'card-heading'})
            Service_name = program_heading_element.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click click-cookie'}).text
            print('Service name: ', Service_name)
    
            # Extract program URL
            program_url_element = program_heading_element.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click click-cookie'})
            program_url = program_url_element['href'] if program_url_element else None
            Service_url = "https://www.auntbertha.com/"+program_url
            print('URL: ', Service_url)
            
            is_reviewed = False

            # Reviewed on
            Reviewd_on_element = program_heading_element.find('div', {'class': 'last-reviewed'})
            if Reviewd_on_element:  # Check if the element exists
                Reviewd_on_text = Reviewd_on_element.get_text(strip=True)  # Clean up the text
                match = re.search(r'\d{2}/\d{2}/\d{4}', Reviewd_on_text)
                Reviewd_on = match.group() if match else None
                if Reviewd_on:  # If we found a review date
                    is_reviewed = True  # Set the flag to True
                print("Reviewed on:", Reviewd_on)
            else:
                Reviewd_on = None  # Set to None if not found
                print("No review date found.")
    
            # Access class="program-tags"
            program_tags = li_soup.find('div', {'class': 'program-tags'})

            # Main Services
            main_s = []
            main_service_list = program_tags.find('ul', {'class': 'list-inline'})
            main_service_items = main_service_list.find_all('li')

            for item in main_service_items:
                main_service = item.find('a', {'class': 'loading-on-click activity-log'}).text.strip()
                main_s.append(main_service)

            print("Main Services:", main_s)

            # Try to access the "Other Services" section
            other_service_list = program_tags.find('div', {'class': 'secondary-tags'})
            if other_service_list:
                other_s = []
                other_service_items = other_service_list.find('ul', {'class': 'list-inline'}).find_all('li')

                for item in other_service_items:
                    other_service = item.find('a', {'class': 'loading-on-click activity-log'}).text.strip()
                    other_s.append(other_service)
            else:
                other_s = None

            print("Other Services:", other_s)

            # Access the "Serving" section
            serving_section = program_tags.find('div', {'class': 'attribute-tags'})

            # Initialize a list to store the serving information
            serving_ = []
    
            # Find all the <li> elements within the serving section
            serving_items = serving_section.find_all('li')

            # Iterate through the serving items and extract the text from the <a> elements
            for item in serving_items:
                link = item.find('a', {'class': 'loading-on-click activity-log'})
    
                # Check if the link was found
                if link:  # Only proceed if the link exists
                    serving_text = link.text.strip()
                    serving_.append(serving_text)

            print("Serving:", serving_)

    
            # accessing next-steps-module, extract phone number, location, hours 
    
            next_steps_module = li_soup.find('div', {'class': 'next-steps-module'})

            # Extract phone number
            phone_number_elements = next_steps_module.find_all('span', {'class': 'result-next-step-item'})

            # Also find all 'a' elements with href attributes containing 'tel:'
            tel_link_elements = next_steps_module.find_all('a', href=True)

            phone_number = None  # Initialize phone number as None

            # Extract phone numbers from text-based spans
            for element in phone_number_elements:
                phone_number_text = element.text.strip() if element else None
    
                # Regex to extract digits (handling separators like spaces or hyphens)
                phone_number_matches = re.findall(r'[\d-]+', phone_number_text)
    
                # Join the digits into a single phone number string if matches are found
                if phone_number_matches:
                    phone_number = ''.join(phone_number_matches)
                    break  # Stop once we've found a phone number

            # If no phone number found from text, check the 'tel:' href links
            if not phone_number:
                for element in tel_link_elements:
                    href_value = element['href']
        
                    # Check if the href contains 'tel:' and extract digits
                    if 'tel:' in href_value:
                        phone_number_matches = re.findall(r'[\d-]+', href_value)
                        if phone_number_matches:
                            phone_number = ''.join(phone_number_matches)
                            break  # Stop once we've found a phone number

            # Print the extracted phone number
            if phone_number:
                print("Phone Number:", phone_number)
            else:
                print("No phone number found.")

            # Extract location address
            location_address_element = next_steps_module.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click map-link with-address'})
            location_address = location_address_element.text.strip() if location_address_element else None
            location_address = re.sub(r'\s+', ' ', location_address) if location_address_element else None
            print("Location Address:", location_address)

            # Extract URL
            location_url_map = location_address_element['href'] if location_address_element else None
            print("Location url map:", location_url_map)
            
            hours_info = {
                '24_hour': False,
                'Monday': None,
                'Tuesday': None,
                'Wednesday': None,
                'Thursday': None,
                'Friday': None,
                'Saturday': None,
                'Sunday': None
            }
            
            # Extract hours based on structure
            hours_element = next_steps_module.find('div', {'class': 'office-hours-schedule see-hours-dropdown'})
            if hours_element:
                # Find all the <span> elements within the hours_element
                day_spans = hours_element.find_all('span')

                # Ensure that we have even pairs of day and corresponding hours
                if len(day_spans) % 2 == 0:
                    # Iterate through the spans in pairs (day and its corresponding hours)
                    for i in range(0, len(day_spans), 2):
                        day_span = day_spans[i]          # This is the day (e.g., "Monday:")
                        hours_span = day_spans[i + 1]    # This is the corresponding hours (e.g., "Closed" or time)

                        # Extract the day name and remove the colon
                        day = day_span.text.strip()[:-1]  # Remove the ':' at the end
                        full_day = day.strip()            # Ensure there's no extra space

                        # Check if the day is valid and exists in hours_info
                        if full_day in hours_info:
                            # Check if the corresponding hours span indicates "Closed"
                            if 'Closed' in hours_span.text:
                                hours_info[full_day] = 'Closed'
                            else:
                                # Extract hours for non-closed days
                                hours_info[full_day] = hours_span.text.strip()
                else:
                    print("Unexpected hours format. Ensure that each day has corresponding hours.")

            # Handle 24-hour information if it exists
            else:
                hours_element = next_steps_module.find('span', {'class': 'result-geo-hours'})
                if hours_element:
                    # Get 24-hour text strip
                    hours_text = hours_element.get_text(strip=True)

                    # Assuming the presence of this text means 24-hour operation
                    if '24' in hours_text or '24-hour' in hours_text.lower():
                        hours_info['24_hour'] = True
                    else:
                        print("Unable to determine if the office is 24 hours.")
                else:
                    print("Hours information not available.")

            # Output the result
            print(hours_info)
    
    
            # Loop through the found <li> elements and extract labels and values
            Extra_element = li_element.find('div', {'class': 'panel-wrapper more-info-panel'})
            elig = []
            eligibility_rules_element = Extra_element.find('div', {'class': 'eligibility-rules'})
    
            # Check if eligibility_rules_element is found
            if eligibility_rules_element:
                # Check if eligibility_rules_element contains a list (ul)
                ul_element = eligibility_rules_element.find('ul')
        
                if ul_element:
                    # If it contains a list, extract list items and store them in a flat list
                    eligibility_list = [li.text.strip() for li in ul_element.find_all('li')]
                    elig.extend(eligibility_list)  # Use extend to add elements to the list directly
                    #print("Eligibility:", eligibility_list)
                else:
                    # If it doesn't contain a list, store the text as is
                    eligibility_text = eligibility_rules_element.text.strip()
                    #print("Eligibility:", [eligibility_text])  # Wrap in a list to maintain consistency
                    elig.append(eligibility_text)

            else:
                # Skip if 'eligibility-rules' class is not found
                pass
    
        
            print("Eligibility:", elig)
    
            # Extract Availability
            availability_element = Extra_element.find('strong', {'data-translate': 'Availability'})
            
            if availability_element:
                availability = availability_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            else:
                availability = "Not specified"

            # Extract Description
            description_element = Extra_element.find('strong', {'data-translate': 'Description'})
            description = description_element.find_next('div', {'class': 'col-md-10'}).text.strip()

            # Extract Languages
            languages_element = Extra_element.find('strong', {'data-translate': 'Languages'})
            languages = languages_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            languages_data = [lang.strip() for lang in languages.split(',')]
            
            # Extract Cost
            cost_element = Extra_element.find('strong', string='Cost:')
            
            if cost_element:  # Check if cost_element exists
                cost = cost_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            else:
                cost = "Not specified"  # Default value if cost_element is not found
            
            # Extract Website URLs if they exist, or set them to None
            website_element = Extra_element.find('div', {'data-translation': 'Website'})
            website_url = website_element.find_next('a', {'class': 'activity-log descriptionProgramWebsite'})['href'] if website_element else None

            # Extract Facebook and Twitter URLs if they exist, or set them to None
            facebook_element = Extra_element.find('strong', {'data-translate': 'Facebook'})
            facebook_url = facebook_element.find_next('a', {'class': 'activity-log descriptionProgramFacebook'})['href'] if facebook_element else None

            twitter_element = Extra_element.find('strong', {'data-translate': 'Twitter'})
            twitter_url = twitter_element.find_next('a', {'class': 'activity-log descriptionProgramTwitter'})['href'] if twitter_element else None

            # Extract Coverage Area
            coverage_element = Extra_element.find('strong', {'data-translate': 'Coverage Area'})
            coverage = coverage_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            
            # Initialize latitude and longitude as None by default
            latitude = None
            longitude = None
            zipcode = None

            # Find the element with class "office-hour-address"
            location_element = Extra_element.find('div', {'class': 'office-hours-address _js_address address notranslate'})

            # Check if the element exists and has the required attributes
            if location_element:
                latitude = location_element['data-latitude'] if location_element.has_attr('data-latitude') else None
                longitude = location_element['data-longitude'] if location_element.has_attr('data-longitude') else None
                print("Latitude:", latitude)
                print("Longitude:", longitude)
                
                # Extract all text within location_element
                address_text = location_element.get_text(separator=" ").strip()  # Get all text as a single string

                # Use regular expression to search for the ZIP code pattern anywhere in the text
                zip_matches = re.findall(r'(?<!\d)(\b\d{5}\b)(?!\d)', address_text)
                zipcode = int(zip_matches[-1]) if zip_matches else None
                print("ZIP Code:", zipcode)
                
            Google_Reviews = None
            Service_Type = None

            # Print or use the extracted values as needed
            print("Availability:", availability)
            print("Description:", description)
            print("Languages:", languages_data)
            print("Cost:", cost)
            print("Facebook URL:", facebook_url)
            print("Twitter URL:", twitter_url)
            print("Coverage Area:", coverage)

            data.append([
            Service_name,
            Service_url,
            main_s,
            other_s,
            serving_,
            phone_number,
            website_url,
            location_address,
            location_url_map,
            elig,
            availability,
            description,
            languages_data,
            cost,
            is_reviewed,
            facebook_url,
            twitter_url,
            coverage,
            latitude,
            longitude,
            zipcode,
            hours_info['24_hour'],
            hours_info['Monday'],      
            hours_info['Tuesday'],     
            hours_info['Wednesday'],   
            hours_info['Thursday'],   
            hours_info['Friday'],     
            hours_info['Saturday'],  
            hours_info['Sunday'],
            Google_Reviews,
            Service_Type])
            print("***************************************************************************")

Service name:  Community Center
URL:  https://www.auntbertha.com//the-salvation-army-of-santa-monica--santa-monica-ca--community-center/4865175637196800?postal=90012
Reviewed on: 12/13/2024
Main Services: ['food pantry', 'recreation', 'support network', 'one-on-one support', 'spiritual support']
Other Services: None
Serving: ['anyone in need', 'all ages']
Phone Number: 310-451-1358
Location Address: 1533 4th St, Santa Monica, CA 90401
Location url map: https://www.google.com/maps/?q=1533+4th+St,+Santa+Monica,+CA+90401/
{'24_hour': False, 'Monday': '8:00 AM - 5:00 PM  PST', 'Tuesday': '8:00 AM - 5:00 PM  PST', 'Wednesday': '8:00 AM - 5:00 PM  PST', 'Thursday': '8:00 AM - 5:00 PM  PST', 'Friday': '8:00 AM - 5:00 PM  PST', 'Saturday': '12:00 AM - 12:00 AM  PST', 'Sunday': '12:00 AM - 12:00 AM  PST'}
Eligibility: []
Latitude: 34.014525
Longitude: -118.4924575
ZIP Code: 90401
Availability: available
Description: The Santa Monica Corps Community Center is a resource of Christian ministry for

Service name:  Food Distribution Program
URL:  https://www.auntbertha.com//los-angeles-housing-partnership%252C-inc.-%2528lahp%2529--los-angeles-ca--food-distribution-program/5262529185972224?postal=90012
Reviewed on: 12/15/2024
Main Services: ['food pantry']
Other Services: None
Serving: ['all ages', 'individuals', 'families', 'homeless', 'low-income']
Phone Number: 213-629-9172109
Location Address: 2614 West 7th Street, Los Angeles, CA 90057
Location url map: https://www.google.com/maps/?q=2614+West+7th+Street,+Los+Angeles,+CA+90057/
{'24_hour': False, 'Monday': 'Closed', 'Tuesday': 'Closed', 'Wednesday': 'Closed', 'Thursday': 'Closed', 'Friday': '1:00 PM - 4:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['Must meet income guidelines.']
Latitude: 34.0586595
Longitude: -118.2828031
ZIP Code: 90057
Availability: available
Description: The Los Angeles Housing Partnership Food Distribution Program regularly distributes food to residents and communities throughout LA

Service name:  Food Pantry
URL:  https://www.auntbertha.com//world-harvest-food-bank--los-angeles-ca--food-pantry/5195422603149312?postal=90012
Reviewed on: 12/15/2024
Main Services: ['food pantry']
Other Services: None
Serving: ['anyone in need', 'all ages', 'low-income']
Phone Number: 213-746-2227
Location Address: 3100 Venice Boulevard, Los Angeles, CA 90019
Location url map: https://www.google.com/maps/?q=3100+Venice+Boulevard,+Los+Angeles,+CA+90019/
{'24_hour': False, 'Monday': '8:00 AM - 6:00 PM  PST', 'Tuesday': '8:00 AM - 6:00 PM  PST', 'Wednesday': '8:00 AM - 6:00 PM  PST', 'Thursday': '8:00 AM - 6:00 PM  PST', 'Friday': '8:00 AM - 6:00 PM  PST', 'Saturday': '8:00 AM - 3:00 PM  PST', 'Sunday': 'Closed'}
Eligibility: ['Must make a donation of $40 or donate your time as a volunteer for one part time day of 4 hours.']
Latitude: 34.0432466
Longitude: -118.3181131
ZIP Code: 90019
Availability: available
Description: World Harvest provides food, basic staples and living essentials t

Service name:  Food Pantry
URL:  https://www.auntbertha.com//salvation-army-southern-california---inglewood-citadel-corps--inglewood-ca--food-pantry/4758736683925504?postal=90012
Reviewed on: 12/13/2024
Main Services: ['emergency food', 'food pantry']
Other Services: None
Serving: ['anyone in need', 'all ages']
No phone number found.
Location Address: 324 E Queen St, Inglewood, CA 90301
Location url map: https://www.google.com/maps/?q=324+E+Queen+St,+Inglewood,+CA+90301/
{'24_hour': False, 'Monday': '1:00 PM - 3:00 PM  PST', 'Tuesday': '1:00 PM - 3:00 PM  PST', 'Wednesday': '1:00 PM - 3:00 PM  PST', 'Thursday': '1:00 PM - 3:00 PM  PST', 'Friday': '1:00 PM - 3:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: []
Latitude: 33.9626964
Longitude: -118.3508704
ZIP Code: 90301
Availability: available
Description: Our food pantry provides free food to feed families and individuals in need. We supply groceries for residents that do not have enough food because they lack the r

Service name:  Food Pantry
URL:  https://www.auntbertha.com//st-john%2527s-cathedral--los-angeles-ca--food-pantry/4721561814171648?postal=90012
Reviewed on: 12/13/2024
Main Services: ['food pantry']
Other Services: None
Serving: ['anyone in need', 'all ages', 'individuals', 'families']
No phone number found.
Location Address: 514 West Adams Boulevard, Los Angeles, CA 90007
Location url map: https://www.google.com/maps/?q=514+West+Adams+Boulevard,+Los+Angeles,+CA+90007/
{'24_hour': False, 'Monday': 'Closed', 'Tuesday': 'Closed', 'Wednesday': 'Closed', 'Thursday': 'Closed', 'Friday': 'Closed', 'Saturday': '8:30 AM - 10:30 AM  PST', 'Sunday': 'Closed'}
Eligibility: ['Anyone can access this program.']
Latitude: 34.0274558
Longitude: -118.2755905
ZIP Code: 90007
Availability: available
Description: St John's Cathedral food pantry serves everyone from low income families, single parents, senior citizens, unemployed individuals, disabled veterans, working poor, and anyone else in need of thei

...

Service name:  Food Pantry Locator
URL:  https://www.auntbertha.com//los-angeles-regional-food-bank--vernon-ca--food-pantry-locator/5206662112346112?postal=90012
Reviewed on: 12/15/2024
Main Services: ['emergency food', 'food pantry', 'navigating the system']
Other Services: None
Serving: ['anyone in need', 'all ages']
No phone number found.
Location Address: None
Location url map: None
{'24_hour': False, 'Monday': '8:30 AM - 5:00 PM  PST', 'Tuesday': '8:30 AM - 5:00 PM  PST', 'Wednesday': '8:30 AM - 5:00 PM  PST', 'Thursday': '8:30 AM - 5:00 PM  PST', 'Friday': '8:30 AM - 5:00 PM  PST', 'Saturday': '9:00 AM - 12:00 PM  PST', 'Sunday': 'Closed'}
Eligibility: []
Availability: available
Description: Los Angeles Regional Food Bank offers the Food Pantry Locator to help families find their local food pantry. Services include:- Help finding foodEnter your address on our website to find your local food pantry. Note: Pantry distribution dates and times are subject to change.
Languages: ['Engl

Service name:  Calvary Light Resource Center
URL:  https://www.auntbertha.com//calvary-light-christian-center--los-angeles-ca--calvary-light-resource-center/5797766144589824?postal=90012
Reviewed on: 12/17/2024
Main Services: ['food pantry']
Other Services: None
Serving: ['anyone in need', 'all ages', 'individuals', 'families']
No phone number found.
Location Address: 1101 Lakme Avenue, Los Angeles, CA 90744
Location url map: https://www.google.com/maps/?q=1101+Lakme+Avenue,+Los+Angeles,+CA+90744/
{'24_hour': False, 'Monday': 'Closed', 'Tuesday': 'Closed', 'Wednesday': '10:30 AM - 1:30 PM', 'Thursday': 'Closed', 'Friday': 'Closed', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['Anyone can access this program.']
Latitude: 33.7850459
Longitude: -118.2601068
ZIP Code: 90744
Availability: available
Description: Calvary Light Resource Center provides a walk-up and drive-through food pantry to anyone in need of food on a first come first serve basis.This program provides:- Food to 

Service name:  San Gabriel Valley Service Center
URL:  https://www.auntbertha.com//los-angeles-county-workforce-development%252C-aging-%2526-community-services-%2528wdacs%2529--south-el-monte-ca--san-gabriel-valley-service-center/5472387637182464?postal=90012
Reviewed on: 12/16/2024
Main Services: ['emergency food', 'food pantry', 'exercise & fitness', 'recreation', 'navigating the system', 'help fill out forms', 'support network', 'more education', 'skills & training', 'translation & interpretation']
Other Services: None
Serving: ['all ages', 'individuals', 'families', 'low-income']
Phone Number: 626-575-5431
Location Address: 1441 Santa Anita Avenue, South El Monte, CA 91733
Location url map: https://www.google.com/maps/?q=1441+Santa+Anita+Avenue,+South+El+Monte,+CA+91733/
{'24_hour': False, 'Monday': '8:00 AM - 6:30 PM  PST', 'Tuesday': '8:00 AM - 6:30 PM  PST', 'Wednesday': '8:00 AM - 6:30 PM  PST', 'Thursday': '8:00 AM - 6:30 PM  PST', 'Friday': '8:00 AM - 6:30 PM  PST', 'Saturday

Service name:  Food Pantry
URL:  https://www.auntbertha.com//toberman-neighborhood-center--los-angeles-ca--food-pantry/5151488823263232?postal=90012
Reviewed on: 12/14/2024
Main Services: ['emergency food', 'food delivery', 'food pantry']
Other Services: None
Serving: ['anyone in need', 'all ages', 'individuals', 'families', 'homeless', 'near homeless', 'low-income']
Phone Number: 310-832-1145106
Location Address: 131 North Grand Avenue, Los Angeles, CA 90731
Location url map: https://www.google.com/maps/?q=131+North+Grand+Avenue,+Los+Angeles,+CA+90731/
{'24_hour': False, 'Monday': '8:30 AM - 5:30 PM  PST', 'Tuesday': '8:30 AM - 8:00 PM  PST', 'Wednesday': '8:30 AM - 5:30 PM  PST', 'Thursday': '8:30 AM - 8:00 PM  PST', 'Friday': '8:30 AM - 5:30 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: []
Latitude: 33.7438507
Longitude: -118.2903072
ZIP Code: 90731
Availability: available
Description: Toberman Neighborhood Center provides food assistance to those in need through 

Service name:  Basic Needs Assistance
URL:  https://www.auntbertha.com//lady-bug-ministry-inc.--compton-ca--basic-needs-assistance/6397099490934784?postal=90012
Reviewed on: 12/11/2024
Main Services: ['food pantry', 'nutrition education']
Other Services: None
Serving: ['all ages', 'individuals', 'families', 'with children']
Phone Number: 323-751-3748
Location Address: , Compton, CA 90220
Location url map: https://www.google.com/maps/?q=/
{'24_hour': False, 'Monday': '10:00 AM - 3:00 PM  PST', 'Tuesday': '10:00 AM - 3:00 PM  PST', 'Wednesday': 'Closed', 'Thursday': '10:00 AM - 1:00 PM  PST', 'Friday': 'Closed', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['This program serves men, women and children who are in need of nutritious food, clothing, furnishing and easy access to educational services.']
Availability: available
Description: Lady Bug Ministry helps assist men, women and children with a variety of basic needs services. This program helps individuals and families live 

In [3]:
# Save to CSV file 
#"FindHelp_extracted_data_riv_shelter.csv" is the name for the shelter csv for riv
#"FindHelp_extracted_data_riv_food_pantry.csv" is the name for the food pantry csv for riv
#"FindHelp_extracted_data_riv_mental_health.csv" is the name for the food pantry csv for riv
#"FindHelp_extracted_data_downtown_la_food_pantry.csv" is the name for the food pantry csv for downtown la
#"FindHelp_extracted_data_downtown_la_shelter.csv"
#FindHelp_extracted_data_downtown_la_mental_health.csv
csv_filename = "FindHelp_extracted_data_irvine_food_pantry.csv"

with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # The header 
    header = [
        "Service_name",
        "Service_url",
        "Main_Services",
        "Other_Services",
        "Serving",
        "Phone_Number",
        "Website",
        "Location_Address",
        "Location_URL_Map",
        "Eligibility",
        "Availability",
        "Description",
        "Languages",
        "Cost",
        "Google_Review",
        "Facebook_URL",
        "Twitter_URL",
        "Coverage",
        "Latitude",
        "Longitude",
        "Zipcode",
        "24hour",
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
        "Friday",
        "Saturday",
        "Sunday",
        "Google_Reviews",
        "Service_Type"
    ]
    csv_writer.writerow(header)
    
    csv_writer.writerows(data)

In [4]:
#"FindHelp_extracted_data_riv_shelter.csv" is the name for the shelter csv
#"FindHelp_extracted_data_riv_food_pantry.csv" is the name for the food pantry csv
#"FindHelp_extracted_data_riv_mental_health.csv" is the name for the food pantry csv
#"FindHelp_extracted_data_downtown_la_food_pantry.csv"
#"FindHelp_extracted_data_downtown_la_shelter.csv"
#"FindHelp_extracted_data_downtown_la_mental_health.csv"
#"FindHelp_extracted_data_irvine_food_pantry.csv"
#"FindHelp_extracted_data_irvine_shelter.csv"
#"FindHelp_extracted_data_irvine_mental_health.csv"
df = pd.read_csv("FindHelp_extracted_data_irvine_food_pantry.csv")

#riv_zipcodes = [92501, 92502, 92503, 92504, 92505, 92506, 92507, 92508, 92513, 92514, 92516, 92517, 92521, 92522]

downtown_la_zipcodes = [90012, 90013, 90014, 90015, 90017, 90021, 90071, 90079]

#irvine_zipcodes = [92602, 92603, 92604, 92606, 92612, 92614, 92617, 92618, 92620]

#Philadelphia_zipcodes = [19131, 19132, 19133, 19134, 19135, 19136, 19137, 19138, 19139, 19140, 19141, 19142, 19143, 19144, 19145, 19146, 19147, 19148, 19149, 19150, 19151, 19152, 19153, 19154]

filtered_df = df[df['Zipcode'].isin(downtown_la_zipcodes) | (df['Location_Address'].isna() & df['Longitude'].isna() & df['Latitude'].isna())]

filtered_df = filtered_df.drop_duplicates(subset = ['Website', 'Service_name'])

filtered_df.to_csv("FindHelp_extracted_data_irvine_food_pantry.csv", index=False, encoding='utf-8')

filtered_df

,Service_name,Service_url,Main_Services,Other_Services,Serving,Phone_Number,Website,Location_Address,Location_URL_Map,Eligibility,...,24hour,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Google_Reviews,Service_Type
26,FamilySource Center,https://www.auntbertha.com//new-economics-for-...,"['food pantry', 'understand government program...",NaN,"['all ages', 'families', 'with children', 'sin...",213-483-2060,https://neweconomicsforwomen.org/familysource-...,NaN,NaN,['This program serves assist low-income reside...,...,False,9:00 AM - 5:00 PM PST,9:00 AM - 5:00 PM PST,9:00 AM - 5:00 PM PST,9:00 AM - 5:00 PM PST,9:00 AM - 5:00 PM PST,Closed,Closed,NaN,NaN
27,Homeless Outreach Program,https://www.auntbertha.com//my-friends-house-l...,"['food pantry', 'clothing', 'personal care ite...",['spiritual support'],"['all ages', 'homeless']",NaN,http://www.myfriendshousela.org/about-us.html,"540 San Pedro Street, Los Angeles, CA 90013",https://www.google.com/maps/?q=540+San+Pedro+S...,[],...,False,Closed,Closed,12:00 PM - 2:00 PM PST,Closed,Closed,Closed,Closed,NaN,NaN
38,Pop-Up Pantries,https://www.auntbertha.com//westside-food-bank...,['food pantry'],NaN,"['anyone in need', 'all ages', 'individuals', ...",NaN,https://www.wsfb.org/our-services/need-food-help/,NaN,NaN,['Anyone can access this program.'],...,False,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM,Closed,Closed,NaN,NaN
45,Food Pantry,https://www.auntbertha.com//wilshire-presbyter...,['food pantry'],NaN,"['anyone in need', 'all ages', 'individuals', ...",213-387-5387,https://www.wpcla.org/,NaN,NaN,['Anyone can access this program.'],...,False,8:00 AM - 10:00 AM PST,Closed,Closed,Closed,Closed,Closed,Closed,NaN,NaN
49,Food Distribution Program,https://www.auntbertha.com//prep-ed-programs%2...,['food pantry'],NaN,"['all ages', 'individuals', 'families', 'homel...",NaN,https://prepedprograms.org/services-programs,NaN,NaN,['This program serves low-income and homeless ...,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,Thursday Food Bank,https://www.auntbertha.com//central-city-neigh...,['food pantry'],NaN,"['families', 'all ages']",213-482-8618,http://laccnp.org,"501 South Bixel Street, Los Angeles, CA 90017",https://www.google.com/maps/?q=501+South+Bixel...,[],...,False,Closed,Closed,Closed,9:00 AM - 1:00 PM PST,Closed,Closed,Closed,NaN,NaN
52,Food and Clothing Giveaways,https://www.auntbertha.com//stay-free-ministri...,"['food pantry', 'clothing']",NaN,"['anyone in need', 'all ages', 'individuals', ...",310-966-0162,https://stayfreeministries.org/about-us,"545 S San Pedro St, Los Angeles, CA 90013",https://www.google.com/maps/?q=545+S+San+Pedro...,['Anyone can access this program.'],...,False,Closed,Closed,Closed,Closed,Closed,8:00 AM - 5:00 PM PST,Closed,NaN,NaN
54,Vida Sana,https://www.auntbertha.com//pico-union-project...,['food pantry'],NaN,"['anyone in need', 'all ages', 'individuals', ...",NaN,https://www.picounionproject.org/,"1153 Valencia Street, Los Angeles, CA 90015",https://www.google.com/maps/?q=1153+Valencia+S...,[],...,False,Closed,Closed,Closed,Closed,8:30 AM - 1:00 PM,Closed,Closed,NaN,NaN
60,Food Pantry Locator,https://www.auntbertha.com//los-angeles-region...,"['emergency food', 'food pantry', 'navigating ...",NaN,"['anyone in need', 'all ages']",NaN,https://www.lafoodbank.org/find-food/pantry-lo...,NaN,NaN,[],...,False,8:30 AM - 5:00 PM PST,8:30 AM - 5:00 PM PST,8:30 AM - 5:00 PM PST,8:30 AM - 5:00 PM PST,8:30 AM - 5:00 PM PST,9:00 AM - 12:00 PM PST,Closed,NaN,NaN
62,Community Center,https://www.auntbertha.com//salvation-army-sou...,"['help pay for utilities', 'food pantry', 'mea...",NaN,"['anyone in need', 'all ages', 'individuals', ...",213-896-9178,http://losangelescentral.salvationarmy.org/,"1532 West 11th Street, Los Angeles, CA 90015",https://www.google.com/maps/?q=1532+West+11th+...,[],...,False,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PS